Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import time

In [3]:
pickle_file = '/home/mattw/data/other/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [9]:
import time
num_steps = 1001

start = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    if (step % 100 == 0):
      _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
    else:
      session.run(optimizer, feed_dict=feed_dict)
        
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
end = time.time()
print(end-start)

Initialized
Minibatch loss at step 0: 2.470639
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 100: 0.850812
Minibatch accuracy: 81.2%
Validation accuracy: 68.4%
Minibatch loss at step 200: 0.367718
Minibatch accuracy: 93.8%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.693718
Minibatch accuracy: 75.0%
Validation accuracy: 79.4%
Minibatch loss at step 400: 0.583917
Minibatch accuracy: 75.0%
Validation accuracy: 79.5%
Minibatch loss at step 500: 0.592468
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 600: 0.513380
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 700: 0.208714
Minibatch accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch loss at step 800: 0.607996
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Minibatch loss at step 900: 0.288440
Minibatch accuracy: 100.0%
Validation accuracy: 83.3%
Minibatch loss at step 1000: 0.800700
Minibatch accuracy: 68.8%
Validation accuracy: 83.0

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  """Notes
  1. ReLUs and max-pooling operations commute (as do any monotonically increasing
      activation functions) so it is best to downsample with pooling first, and
      then apply the activation function
  """
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

num_steps = 1001

start = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    if (step % 100 == 0):
      _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
    else:
      session.run(optimizer, feed_dict=feed_dict)
        
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
end = time.time()
print(end-start)



Initialized
Minibatch loss at step 0: 3.123348
Minibatch accuracy: 6.2%
Validation accuracy: 10.4%
Minibatch loss at step 100: 0.843408
Minibatch accuracy: 62.5%
Validation accuracy: 69.2%
Minibatch loss at step 200: 0.360466
Minibatch accuracy: 93.8%
Validation accuracy: 78.7%
Minibatch loss at step 300: 0.529385
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 400: 0.561102
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 500: 0.409809
Minibatch accuracy: 93.8%
Validation accuracy: 81.7%
Minibatch loss at step 600: 0.460505
Minibatch accuracy: 81.2%
Validation accuracy: 81.5%
Minibatch loss at step 700: 0.156104
Minibatch accuracy: 93.8%
Validation accuracy: 82.6%
Minibatch loss at step 800: 0.451150
Minibatch accuracy: 87.5%
Validation accuracy: 84.0%
Minibatch loss at step 900: 0.389487
Minibatch accuracy: 93.8%
Validation accuracy: 84.5%
Minibatch loss at step 1000: 0.563965
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [7]:
batch_size = 32
patch_size1 = 5
patch_size2 = 5
depth1 = 32
depth2 = 32
num_hidden1 = 512
num_hidden2 = 256
hidden_input_num = image_size // 4 * image_size // 4 * depth2

graph = tf.Graph()

with graph.as_default():

  # constants that have yet to be defined
  keep_prob = tf.placeholder(tf.float32)

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size1, patch_size1, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size2, patch_size2, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [hidden_input_num, num_hidden1], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden1]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden1, num_hidden2], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden2]))
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden2, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  """Notes
  1. ReLUs and max-pooling operations commute (as do any monotonically increasing
      activation functions) so it is best to downsample with pooling first, and
      then apply the activation function
  """
  def model(data, keep_prob):
    # Layer 1
    conv = tf.nn.conv2d(data, layer1_weights, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer1_biases)
    
    # Layer 2
    conv = tf.nn.conv2d(hidden, layer2_weights, strides=[1, 1, 1, 1], padding='SAME')
    pool = tf.nn.max_pool(conv, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(pool + layer2_biases)
    
    # Layer 3
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    reshape = tf.nn.dropout(reshape, keep_prob)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    
    # Layer 4
    hidden = tf.nn.dropout(hidden, keep_prob)
    hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)

    # Layer 5
    hidden = tf.nn.dropout(hidden, keep_prob)
    return tf.matmul(hidden, layer5_weights) + layer5_biases

  # Training computation.
  logits = model(tf_train_dataset, keep_prob)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels)) + 0.001*(
        tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer5_weights))
    
  # Optimizer - learning rate decay
  global_step = tf.Variable(0, trainable=False)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.05, global_step, 2000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Optimizer - no learning rate decay
#   learning_rate = tf.Variable(0.1, trainable=False)
#   optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

num_steps = 300001

start = time.time()
with tf.Session(graph=graph) as session:
    
  tf.initialize_all_variables().run()
  print('Initialized')
  
  # run optimization
  for step in range(num_steps):
    
    # get batch of data
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0}
    
    # output updates periodically
    if (step % 10000 == 0):
      _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
    
    else:
      session.run(optimizer, feed_dict=feed_dict)

  end = time.time()
  
  # print final test accuracy of model
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

# print model-fitting time
print('Time to fit model: %g s' % (end-start))



Initialized
Minibatch loss at step 0: 19.948608
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
Minibatch loss at step 10000: 1.941832
Minibatch accuracy: 87.5%
Validation accuracy: 90.3%
Minibatch loss at step 20000: 0.988426
Minibatch accuracy: 90.6%
Validation accuracy: 91.5%
Minibatch loss at step 30000: 0.553779
Minibatch accuracy: 96.9%
Validation accuracy: 91.5%
Minibatch loss at step 40000: 0.503178
Minibatch accuracy: 93.8%
Validation accuracy: 91.7%
Minibatch loss at step 50000: 0.346040
Minibatch accuracy: 96.9%
Validation accuracy: 92.0%
Minibatch loss at step 60000: 0.312025
Minibatch accuracy: 93.8%
Validation accuracy: 91.8%
Minibatch loss at step 70000: 0.371550
Minibatch accuracy: 93.8%
Validation accuracy: 91.9%
Minibatch loss at step 80000: 0.249024
Minibatch accuracy: 96.9%
Validation accuracy: 92.0%
Minibatch loss at step 90000: 0.214056
Minibatch accuracy: 96.9%
Validation accuracy: 92.0%
Minibatch loss at step 100000: 0.185914
Minibatch accuracy: 100.0%
Valid

# RESULTS
### 2Conv/MaxPool - 3Full
5x5x1x32 (s=1, 2x2 s=1) | 5x5x1x32 (s=1, 2x2 s=1) | 512 hid | 256 hid | 10 hid
- **91.8 / 96.6** (keep_prob = 1.0, l2_pen = 0.001, 100000 steps)
- 90.1 / 95.3 (keep_prob = 1.0, l2_pen = 0.001, 10000 steps)

5x5x1x16 (s=1, 2x2 s=1) | 5x5x1x16 (s=1, 2x2 s=1) | 512 hid | 256 hid | 10 hid
- 86.4 / 92.1 (keep_prob = 1.0, l2_pen = 0.0001, 10000 steps)
- 88.9 / 94.2 (keep_prob = 1.0, l2_pen = 0.01, 10000 steps)
- 89.6 / 94.8 (keep_prob = 1.0, l2_pen = 0.001, 10000 steps)
- 88.3 / 94.3 (keep_prob = 0.5, 10000 steps)
- 86.9 / 92.9 (keep_prob = 0.5, 10000 steps)
- 89.8 / 95.0 (learning rate starts at 0.05; keep_prob = 1.0)

### 2Conv/MaxPool - 2Full
5x5x1x16 (s=1, 2x2 s=1) | 5x5x1x16 (s=1, 2x2 s=1) | 128 hid | 10 hid
- 88.4 / 94.6 (328.84s) (512 hidden units, keep_prob = 0.5)
- 88.4 / 94.2 (308.29s) (keep_prob = 0.5)
- 90.4 / 95.5 (keep_prob = 1.0)

5x5x1x16 (s=1, 2x2 s=1) | 5x5x1x16 (s=1, 2x2 s=1) | 64 hid | 10 hid
- 10.0 / 10.0 (297.53s) (back to 5x5, learning rate decay starting with 0.5)
- 89.2 / 94.8 (169.78s) (5x5 -> 3x3 for both conv layers) (still increasing; this might need more steps)
- 90.1 / 95.7 (296.41s) (10000 steps, learning rate decay)
- 89.5 / 95.4 (442.06s) (1000->10000 steps)
- 86.6 / 92.9 (047.45s) (learning rate 0.05->0.10)
- 85.3 / 91.6 (047.32s) (batch size 16->32)
- 84.6 / 91.1 (036.48s)

### 2Conv - 2Full
5x5x1x16, str=2 | 5x5x1x16, str=2 | 64 hid | 10 hid - batch_size=16, num_steps=1000, rate=0.05
- 82.0 / 89.2 (11.87s)
